In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from faker import Faker
from datetime import datetime

pd.set_option("display.max_colwidth", -1)

/tmp/ipykernel_149349/545795100.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [2]:
df_dic_influd21 = pd.read_excel("dicionario_sindrome_respiratoria.xlsx")
dic_columns_raw = pd.Series( list( df_dic_influd21.DBF ) ).dropna().to_numpy()

# Organiza os nomes das colunas separando em novos nomes aqueles campos separados por " OU "
dic_columns = []
for col in dic_columns_raw:
    col = col.upper()
    if(" OU " in col):
        col = col.split(" OU ")
        dic_columns.append(col[0])
        dic_columns.append(col[1])
    else: 
        dic_columns.append(col.replace(" OU", ""))
dic_columns = np.array(dic_columns)

In [3]:
df_influd21 = pd.read_csv("INFLUD21-09-06-2022.csv", on_bad_lines='skip', sep = ";")
df_influd21.head(2)

/tmp/ipykernel_149349/588895592.py:1: DtypeWarning: Columns (15,59,61,62,63,91,93,105,107,114,116,117,118,120,122,143) have mixed types. Specify dtype option on import or set low_memory=False.
  df_influd21 = pd.read_csv("INFLUD21-09-06-2022.csv", on_bad_lines='skip', sep = ";")


,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DOSE_2_COV,DOSE_REF,FAB_COV_1,FAB_COV_2,FAB_COVREF,LOTE_REF,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV,FNT_IN_COV
0,05/01/2021,1,04/01/2021,1,AL,1 MICRORREGIAO DE SAUDE,1533.0,MACEIO,270430,HOSPITAL DA MULHER DRA NISE DA SILVEIRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,06/01/2021,1,03/01/2021,1,MS,EX ARS DE CAMPO GRANDE,1975.0,CAMPO GRANDE,500270,PRONCOR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [4]:
def search_field(df, field, return_index = False):
    '''
    Encontra colunas que contém em parte o texto presente em field
    '''
    if(not return_index):
        return df.loc[:,np.array([field in column for column in df.columns])]
    return np.array([field in column for column in df.columns])

In [5]:
# verifica quais colunas da base fornecida de fato pertencem ao dicionário dos dados oferecido
df_columns_in_dic = np.array([col in dic_columns for col in df_influd21.columns])
dic_in_df_columns = np.array([col in df_influd21.columns for col in dic_columns])

In [6]:
# Tem na base e não tem no dicionário. Estranho...
tem_base_nao_tem_dic = df_influd21.columns[~df_columns_in_dic]

# Tem no dicionário e não tem na base. Provavelmente confidenciais...
tem_dic_nao_tem_base = dic_columns[~dic_in_df_columns] 

In [9]:
df_dic_influd21.loc[df_dic_influd21.DBF.isin(tem_dic_nao_tem_base),["DBF","Descrição","Características"]]

,DBF,Descrição,Características
0,NU_NOTIFIC,Número do registro,"Campo Interno\nNúmero sequencial gerado automaticamente pelo sistema. Utilizar o padrão:\n320120000123\nDígito 1: caracteriza o tipo da ficha (1=SG, 2=SRAG-UTI e 3-SRAG Hospitalizado).\nDígitos 2 a 12: número sequencial gerado automaticamente pelo sistema."
11,TEM_CPF,Informar se o paciente notificado dispõe de Número do Cadastro de Pessoa Física (CPF),"Campo Obrigatório\nSe selecionado “Sim”, preencher campo “CPF”. Se selecionado “Não” preencher CNS. Se o paciente não dispor de CPF é obrigatório o preenchimento do CNS. No caso de pacientes raça/cor indígenas, somente o CNS é considerado como campo obrigatório."
12,NU_CPF,Número do Cadastro de Pessoa Física (CPF) do paciente notificado,"Campo Obrigatório\nQuando preenchido o número do CPF o sistema deverá preencher o Nome, Sexo, Data de Nascimento, Idade, Raça/Cor e o nome da mãe do paciente."
14,NU_CNS,Preencher com o número do Cartão Nacional de Saúde do paciente,Campo Obrigatório
15,NM_PACIENT,Nome completo do,Campo Obrigatório
24,CS_ETINIA,"Nome e código da etnia do paciente, quando indígena.",Campo Essencial\nHabilitado se campo\n11-Raça/Cor for igual a 5-Indígena.
25,POV_CT,Informar se o paciente for membro de algum povo ou comunidade tradicional,Campo Obrigatório
26,TP_POV_CT,Informar o povo ou comunidade tradicional,Campo Obrigatório- Habilitado se campo\n13- É membro de povo ou comunidade tradicional? for igual a 1- Sim
30,NM_MAE_PAC,Nome completo da mãe do paciente (sem abreviações).,Campo Essencial
31,NU_CEP,CEP de residência do paciente.,Campo Essencial\nValidado a partir da tabela de CEP dos Correios.


In [33]:
df_dic_influd21.loc[ df_dic_influd21.DBF in tem_dic_nao_tem_base , : ]

/tmp/ipykernel_145499/410008494.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  df_dic_influd21.loc[ df_dic_influd21.DBF in tem_dic_nao_tem_base , : ]


ValueError: ('Lengths must match to compare', (183,), (30,))

### Mock database

In [ ]:
def create_user(name : str = None, mothers_name : str = None, birth_dt = None):
    print()
    

In [23]:
faker = Faker(["pt-BR"])

k = 10

for i in range(k):
    print(faker.name())

Vicente Dias
Eloah Farias
Srta. Emanuelly Pinto
Gabriel Aragão
Lorenzo Nunes
Gabriela Peixoto
Luiz Otávio Azevedo
Natália Viana
Alexia Rocha
Dra. Helena Costela
